In [ ]:
# 딕셔너리 키로 선언 : 경력 0년=신입 

In [2]:
import openai
import sqlite3

# ✅ OpenAI API 키 설정
openai.api_key = ""

# ✅ SQLite 데이터베이스 연결
db_path = "asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ `duty_analysis` 테이블에서 career 데이터 조회
cursor.execute("SELECT duty, career FROM duty_analysis;")
rows = cursor.fetchall()

# ✅ 직무별 연차별 채용 데이터 조회
cursor.execute("""
    SELECT duty, career, COUNT(*) as job_count
    FROM processing
    GROUP BY duty, career
    ORDER BY duty, career;
""")
career_data = cursor.fetchall()

# ✅ 데이터 변환 (직무별 연차별 데이터 정리)
job_career_dict = {}
for duty, career, job_count in career_data:
    if duty not in job_career_dict:
        job_career_dict[duty] = {}
    job_career_dict[duty][career] = job_count

# ✅ "경력 0년차"를 "신입"으로 변경
for duty in job_career_dict.keys():
    if 0 in job_career_dict[duty]:  # 0년차 데이터가 존재하면
        job_career_dict[duty]["신입"] = job_career_dict[duty].pop(0)  # "신입" 키로 이동 후 기존 0 삭제

# ✅ 직무별 GPT-3.5 요약 생성 및 업데이트
for duty, career in rows:
    if duty not in job_career_dict:
        continue  # 해당 직무에 대한 데이터가 없으면 건너뜀

    # ✅ 프롬프트 생성
    job_details = "\n".join([f"- {key}년차: {value}건" if key != "신입" else f"- 신입: {value}건"
                              for key, value in job_career_dict[duty].items()])
    prompt = f"""
    "{duty}" 직무의 채용 공고 데이터를 분석해주세요.

    [직무별 연차별 채용 공고 개수]
    {job_details}

    위 데이터를 기반으로 다음을 분석해주세요:
    1. 가장 많이 요청된 경력 연차와 해당 채용 공고 수
    2. 두 번째로 많이 요청된 경력 연차와 해당 채용 공고 수
    3. 최소 요구되는 연차
    4. {duty} 직무에서의 경력별 채용 특징 및 트렌드 분석

    ❗ 단, "신입"이라는 표현을 사용하며, "경력 0년차"라는 표현은 하지 마세요.
    ❗ 모든 문장은 "입니다"로 끝나도록 통일해주세요.
    ❗ 설명은 간결하게 요약해서 제공해주세요. 
    """

    try:
        # ✅ GPT-3.5 API 호출
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        gpt3_summary = response.choices[0].message.content.strip()

        # ✅ 기존 career 컬럼 값 가져오기 (이미지 경로 유지)
        cursor.execute("SELECT career FROM duty_analysis WHERE duty = ?;", (duty,))
        existing_career_data = cursor.fetchone()

        if existing_career_data and existing_career_data[0].endswith(".png"):
            image_path = existing_career_data[0]  # 이미지 경로 유지
        else:
            image_path = ""

        # ✅ `career` 값 업데이트 (['이미지 경로', 'GPT-3.5 설명'])
        career_list = [image_path, gpt3_summary] if image_path else [gpt3_summary]
        updated_text = str(career_list)  # 리스트를 문자열로 변환

        cursor.execute(
            "UPDATE duty_analysis SET career = ? WHERE duty = ?;",
            (updated_text, duty)
        )
        conn.commit()

        print(f"✅ {duty} 요약 완료 (GPT-3.5 적용): {gpt3_summary}")

    except Exception as e:
        print(f"❌ {duty} 요약 실패: {e}")

# ✅ DB 연결 종료
conn.close()
print("🎯 모든 직무의 경력 요구 사항 요약이 GPT-3.5로 업데이트되었습니다.")


✅ AI 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 38건입니다.

2. 두 번째로 많이 요청된 경력 연차는 2년차이며, 해당 채용 공고 수는 25건입니다.

3. AI 직무에서의 최소 요구되는 연차는 신입이며, 해당 채용 공고 수는 23건입니다.

4. AI 직무에서는 3년차 경력이 가장 많이 요구되는 트렌드를 보이고 있으며, 신입도 많은 기회가 제공되고 있습니다. 다만 4년차 이상의 경력은 비교적 적게 요구되는 경향이 있습니다.
✅ ERP 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 7년차로, 3건의 채용 공고가 있습니다.
2. 두 번째로 많이 요청된 경력 연차는 3년차로, 3건의 채용 공고가 있습니다.
3. 최소 요구되는 경력은 1년차입니다.
4. ERP 직무에서는 대부분 경력자를 선호하며, 특히 3년차와 7년차의 수요가 높은 편입니다.신입으로 채용되기 어렵지만, 경력을 쌓은 후 높은 수준의 역량이 요구되는 트렌드가 있습니다.
✅ PM 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 22건입니다.

2. 두 번째로 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 14건입니다.

3. 최소 요구되는 연차는 신입이며, 해당 채용 공고 수는 8건입니다.

4. PM 직무에서는 주로 5년차의 경력이 요구되고 있으며, 그 다음으로 3년차가 많이 요구되는 트렌드가 있습니다. 또한 신입으로의 채용 역시 활발히 이루어지고 있습니다.
✅ QA 요약 완료 (GPT-3.5 적용): 1. QA 직무에서 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 수는 39건입니다.

2. 두 번째로 많이 요청된 경력 연차는 5년차로, 해당 채용 공고 수는 20건입니다.

3. QA 직무에서 최소 요구되는 연차는 신입으로, 해당 채용 공고 수는 17건입니다.

4. QA 직무에서는 주로 3년차와 5년차 경력자를 요구하는 경향이

In [10]:
import os
import sqlite3

db_path = "asia (2).db"  # 사용 중인 DB 파일 경로
print(f"📍 현재 DB 파일의 경로: {os.path.abspath(db_path)}")


📍 현재 DB 파일의 경로: c:\Users\asia\Desktop\asia_camp_2024\final_project_asia\3.ETL_DB_Load(Career,Language)\asia (2).db


In [12]:
import sqlite3

db_path = "c:/Users/asia/Desktop/asia_camp_2024/final_project_asia/3.ETL_DB_Load(Career,Language)/asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT duty, career FROM duty_analysis;")
rows = cursor.fetchall()

for duty, career in rows:
    print(f"🔍 {duty}: {career}")

conn.close()


🔍 AI: ['1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 38건입니다.\n\n2. 두 번째로 많이 요청된 경력 연차는 2년차이며, 해당 채용 공고 수는 25건입니다.\n\n3. AI 직무에서의 최소 요구되는 연차는 신입이며, 해당 채용 공고 수는 23건입니다.\n\n4. AI 직무에서는 3년차 경력이 가장 많이 요구되는 트렌드를 보이고 있으며, 신입도 많은 기회가 제공되고 있습니다. 다만 4년차 이상의 경력은 비교적 적게 요구되는 경향이 있습니다.']
🔍 ERP: ['1. 가장 많이 요청된 경력 연차는 7년차로, 3건의 채용 공고가 있습니다.\n2. 두 번째로 많이 요청된 경력 연차는 3년차로, 3건의 채용 공고가 있습니다.\n3. 최소 요구되는 경력은 1년차입니다.\n4. ERP 직무에서는 대부분 경력자를 선호하며, 특히 3년차와 7년차의 수요가 높은 편입니다.신입으로 채용되기 어렵지만, 경력을 쌓은 후 높은 수준의 역량이 요구되는 트렌드가 있습니다.']
🔍 PM: ['1. 가장 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 22건입니다.\n\n2. 두 번째로 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 14건입니다.\n\n3. 최소 요구되는 연차는 신입이며, 해당 채용 공고 수는 8건입니다.\n\n4. PM 직무에서는 주로 5년차의 경력이 요구되고 있으며, 그 다음으로 3년차가 많이 요구되는 트렌드가 있습니다. 또한 신입으로의 채용 역시 활발히 이루어지고 있습니다.']
🔍 QA: ['1. QA 직무에서 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 수는 39건입니다.\n\n2. 두 번째로 많이 요청된 경력 연차는 5년차로, 해당 채용 공고 수는 20건입니다.\n\n3. QA 직무에서 최소 요구되는 연차는 신입으로, 해당 채용 공고 수는 17건입니다.\n\n4. QA 직무에서는 주로 3년차와 5년차 경력자를 요구하는 경향이 있으며, 신입 채용도 활발하게 이루어지고 있습니다. 또한, 4년차 이하의